In [308]:
# To plot inline
%matplotlib inline

# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [309]:
data_train = pd.read_csv('train.csv')
print('Number Of Rows: {}'.format(len(data_train)))
print('--------------------------')
data_train.head()

Number Of Rows: 891
--------------------------


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [310]:
for x in ['Name','Ticket','Cabin','Embarked']: del data_train[x]
data_train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,1,0,3,male,22.0,1,0,7.2500
1,2,1,1,female,38.0,1,0,71.2833
2,3,1,3,female,26.0,0,0,7.9250
3,4,1,1,female,35.0,1,0,53.1000
4,5,0,3,male,35.0,0,0,8.0500


In [311]:
data_train.Sex = data_train.Sex.astype('category').cat.codes
data_train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,1,0,3,1,22.0,1,0,7.2500
1,2,1,1,0,38.0,1,0,71.2833
2,3,1,3,0,26.0,0,0,7.9250
3,4,1,1,0,35.0,1,0,53.1000
4,5,0,3,1,35.0,0,0,8.0500


In [312]:
data_train.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Sex               int8
Age            float64
SibSp            int64
Parch            int64
Fare           float64
dtype: object

In [313]:
print(data_train.isnull().any())
print('----------------------------------')
print('Number Of Rows: {}'.format(len(data_train)))

PassengerId    False
Survived       False
Pclass         False
Sex            False
Age             True
SibSp          False
Parch          False
Fare           False
dtype: bool
----------------------------------
Number Of Rows: 891


In [314]:
print("Average Age Men: %.2f\nAverage Age Women %0.2f"%(data_train[data_train['Sex'] == 0].Age.mean(),data_train[data_train['Sex'] == 1].Age.mean()))
male_mean_age = data_train[data_train['Sex'] == 0].Age.mean()
female_mean_age = data_train[data_train['Sex'] == 1].Age.mean()
# data_train.set_index('Sex',inplace=True)
# data_train.loc[0].Age.fillna(male_mean_age,inplace=True)
# data_train.loc[1].Age.fillna(female_mean_age,inplace=True)
# data_train = data_train.reset_index()
# data_train.isnull().any()
data_train.loc[data_train['Sex'] == 0,'Age'] = data_train.loc[data_train['Sex'] == 0,'Age'].fillna(male_mean_age)
data_train.loc[data_train['Sex'] == 1,'Age'] = data_train.loc[data_train['Sex'] == 1,'Age'].fillna(female_mean_age)
data_train

Average Age Men: 27.92
Average Age Women 30.73


,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,1,0,3,1,22.000000,1,0,7.2500
1,2,1,1,0,38.000000,1,0,71.2833
2,3,1,3,0,26.000000,0,0,7.9250
3,4,1,1,0,35.000000,1,0,53.1000
4,5,0,3,1,35.000000,0,0,8.0500
5,6,0,3,1,30.726645,0,0,8.4583
6,7,0,1,1,54.000000,0,0,51.8625
7,8,0,3,1,2.000000,3,1,21.0750
8,9,1,3,0,27.000000,0,2,11.1333
9,10,1,2,0,14.000000,1,0,30.0708


In [315]:
nmbr = data_train.groupby('Survived').count()['PassengerId']
nmbr

Survived
0    549
1    342
Name: PassengerId, dtype: int64

In [316]:
print('%.2f percent of total people survided.\n%.2f percent people died.'%(nmbr[1]*100/(nmbr[1] + nmbr[0]),nmbr[0]*100/(nmbr[1] + nmbr[0])))

38.38 percent of total people survided.
61.62 percent people died.


In [317]:
data_train['FamilySize'] = data_train.SibSp + data_train.Parch
del data_train['SibSp']
del data_train['Parch']
target = data_train['Survived'].copy() 
data_train.head()

,PassengerId,Survived,Pclass,Sex,Age,Fare,FamilySize
0,1,0,3,1,22.0,7.2500,1
1,2,1,1,0,38.0,71.2833,1
2,3,1,3,0,26.0,7.9250,0
3,4,1,1,0,35.0,53.1000,1
4,5,0,3,1,35.0,8.0500,0


In [318]:
gender_survival = data_train.groupby(['Survived','Sex']).count()['PassengerId']
gender_survival

Survived  Sex
0         0       81
          1      468
1         0      233
          1      109
Name: PassengerId, dtype: int64

In [319]:
pw_survided = gender_survival.iloc[1]*100/(gender_survival.iloc[1] + gender_survival.iloc[0])
pm_died = gender_survival.iloc[2]*100/(gender_survival.iloc[2] + gender_survival.iloc[3])
print('%.2f percent of the total survived are women and %.2f of the total died are men'%(pw_survided,pm_died))

85.25 percent of the total survived are women and 68.13 of the total died are men


In [320]:
del data_train['PassengerId']
del data_train['Survived']
data_train.head()

,Pclass,Sex,Age,Fare,FamilySize
0,3,1,22.0,7.2500,1
1,1,0,38.0,71.2833,1
2,3,0,26.0,7.9250,0
3,1,0,35.0,53.1000,1
4,3,1,35.0,8.0500,0


In [321]:
data_train['Pclass X Fare'] = data_train['Pclass'] * data_train['Fare']
# data_train.drop(['Pclass','Fare'],axis=1,inplace=True)
data_train.head()

,Pclass,Sex,Age,Fare,FamilySize,Pclass X Fare
0,3,1,22.0,7.2500,1,21.7500
1,1,0,38.0,71.2833,1,71.2833
2,3,0,26.0,7.9250,0,23.7750
3,1,0,35.0,53.1000,1,53.1000
4,3,1,35.0,8.0500,0,24.1500


In [322]:
data_train['Pclass X Fare X Age'] = data_train['Pclass X Fare'] * data_train['Age']
data_train.drop(['Pclass X Fare','Age'],axis=1,inplace=True)
data_train

,Pclass,Sex,Fare,FamilySize,Pclass X Fare X Age
0,3,1,7.2500,1,478.500000
1,1,0,71.2833,1,2708.765400
2,3,0,7.9250,0,618.150000
3,1,0,53.1000,1,1858.500000
4,3,1,8.0500,0,845.250000
5,3,1,8.4583,0,779.685534
6,1,1,51.8625,0,2800.575000
7,3,1,21.0750,4,126.450000
8,3,0,11.1333,2,901.797300
9,2,0,30.0708,1,841.982400


In [323]:
X = data_train.as_matrix()
print(X)
y = target.as_matrix()

[[  3.00000000e+00   1.00000000e+00   7.25000000e+00   1.00000000e+00
    4.78500000e+02]
 [  1.00000000e+00   0.00000000e+00   7.12833000e+01   1.00000000e+00
    2.70876540e+03]
 [  3.00000000e+00   0.00000000e+00   7.92500000e+00   0.00000000e+00
    6.18150000e+02]
 ..., 
 [  3.00000000e+00   0.00000000e+00   2.34500000e+01   3.00000000e+00
    1.96387011e+03]
 [  1.00000000e+00   1.00000000e+00   3.00000000e+01   0.00000000e+00
    7.80000000e+02]
 [  3.00000000e+00   1.00000000e+00   7.75000000e+00   0.00000000e+00
    7.44000000e+02]]


In [324]:
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier,GradientBoostingClassifier,VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV,StratifiedKFold,train_test_split,KFold
from sklearn.metrics import f1_score

In [340]:
n_splits = 10

x_tr,x_ts,y_tr,y_ts = train_test_split(X,y)

svc_params = {'C':[1],
              'kernel':['poly']   
                 }

rfc_params = {'n_estimators':[600],
              'min_samples_split':[20],
              'random_state':[15,30]
              'warm_start':(True,False)
             }

logr_params = {
    'penalty':['l2'],
    'max_iter':[100],
    'C':[0.1,0.3,1,3,10]
    
}   

etc_params = {'n_estimators':[20,40],
              'min_samples_split':[10,20,50],
              'random_state':[10,15,30],
              'warm_start':(True,False),
#               'criterion' : ('gini','entropy')
             }

gbc_params = {  'learning_rate':[0.1,0.15],
              'n_estimators':[10,50,80],
              'max_depth':[3,8,10],
              'warm_start':(True,False),
              'subsample' : [0.5,0.8,1.0]
             }
dtc_params = {'criterion':('gini','entropy'),
             'splitter':('best','random'),
              'max_depth':[5,10,15,None],
              'min_samples_split':[2,5,10]
             }
vtc_params = {'voting': ('hard','soft'),
             
      }


SyntaxError: invalid syntax (<ipython-input-340-41987e63e9df>, line 12)

In [341]:
skf = StratifiedKFold(n_splits=n_splits)

# clf1 = SVC()
# clf1 = GridSearchCV(clf1,svc_params,cv=skf,verbose=1,n_jobs=2)
# clf1.fit(x_tr,y_tr)

clf2 = RandomForestClassifier(warm_start=True)
clf2 = GridSearchCV(clf2,rfc_params,cv=skf,verbose=1,n_jobs=2)
clf2.fit(x_tr,y_tr)

# clf2 = LogisticRegression()
# clf2 = GridSearchCV(clf2,logr_params,cv=skf,verbose=2,n_jobs=2)
# clf2.fit(x_tr,y_tr)

# clf3 = ExtraTreesClassifier()
# clf3 = GridSearchCV(clf3,etc_params,cv=skf,verbose=1,n_jobs=2)
# clf3.fit(x_tr,y_tr)

# clf4 = GradientBoostingClassifier()
# clf4 = GridSearchCV(clf4,gbc_params,cv=skf,verbose=1,n_jobs=2)
# clf4.fit(x_tr,y_tr)

# clf5 = DecisionTreeClassifier()
# clf5 = GridSearchCV(clf5,dtc_params,cv=skf,verbose=1,n_jobs=2)
# clf5.fit(x_tr,y_tr)

# models = [x.best_estimator_ for x in [clf1,clf2,clf3,clf4,clf5,clf6]]
# scores = [x.best_score_ for x in [clf1,clf2,clf3,clf4,clf5,clf6]]

    

Fitting 10 folds for each of 2 candidates, totalling 20 fits


[Parallel(n_jobs=2)]: Done  20 out of  20 | elapsed:    8.7s finished


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
       error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=True),
       fit_params={}, iid=True, n_jobs=2,
       param_grid={'n_estimators': [400], 'min_samples_split': [20], 'random_state': [15, 30]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=1)

In [342]:
print(clf2.best_estimator_)
print("Grid Best Score: %.2f"%clf2.best_score_)
training_acc = clf2.best_estimator_.score(x_tr,y_tr)
test_acc = clf2.best_estimator_.score(x_ts,y_ts)
print("Training Score: %.2f\nTesting Score: %.2f" %(training_acc,test_acc))

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=20, min_weight_fraction_leaf=0.0,
            n_estimators=400, n_jobs=1, oob_score=False, random_state=15,
            verbose=0, warm_start=True)
Grid Best Score: 0.84
Training Score: 0.89
Testing Score: 0.84


In [343]:
kf = KFold(n_splits=10)
save = RandomForestClassifier()
test_score = 0
tr_fianl = np.array([])
ts_final = np.array([])
for tr_index,ts_index in kf.split(x_tr,y_tr):
    print(type(tr_index))
    model = RandomForestClassifier(n_estimators=100,n_jobs=2,random_state=35,warm_start=True,criterion='gini',max_features=0.3)
    model.fit(x_tr[tr_index],y_tr[tr_index])
#     test_score = model.score(x_tr[ts_index],y_tr[ts_index])
    if model.score(x_tr[ts_index],y_tr[ts_index]) > test_score:
        save = model
        test_score = model.score(x_tr[ts_index],y_tr[ts_index])
        tr_final = tr_index
        ts_final = ts_index
#     pred = model.predict(x_ts)
    print("Training Score: %.2f\nTesting Score: %.2f" %(model.score(x_tr[tr_index],y_tr[tr_index]),model.score(x_tr[ts_index],y_tr[ts_index])))
    print('-----------------------------------------')
    
# f1_score(y_ts,pred)

<class 'numpy.ndarray'>
Training Score: 0.99
Testing Score: 0.76
-----------------------------------------
<class 'numpy.ndarray'>
Training Score: 0.99
Testing Score: 0.85
-----------------------------------------
<class 'numpy.ndarray'>
Training Score: 0.99
Testing Score: 0.85
-----------------------------------------
<class 'numpy.ndarray'>
Training Score: 0.99
Testing Score: 0.75
-----------------------------------------
<class 'numpy.ndarray'>
Training Score: 0.99
Testing Score: 0.79
-----------------------------------------
<class 'numpy.ndarray'>
Training Score: 0.99
Testing Score: 0.87
-----------------------------------------
<class 'numpy.ndarray'>
Training Score: 0.99
Testing Score: 0.87
-----------------------------------------
<class 'numpy.ndarray'>
Training Score: 0.99
Testing Score: 0.82
-----------------------------------------
<class 'numpy.ndarray'>
Training Score: 0.99
Testing Score: 0.73
-----------------------------------------
<class 'numpy.ndarray'>
Training Scor

In [344]:
save.score(x_ts,y_ts)
# test_score

0.84304932735426008

In [345]:
kf = KFold(n_splits=n_splits)
for tr_index,ts_index in kf.split(X,y):
#     clf6 = VotingClassifier(estimators = [('svc',clf1.best_estimator_),('rfc',clf2.best_estimator_),('etc',clf3.best_estimator_),('gbc',clf4.best_estimator_),('dtc',clf5.best_estimator_)])
    X_train = X[tr_index]
    X_test = X[ts_index]
    y_train = y[tr_index]
    y_test = y[ts_index]
    
    clf2.fit(X_train,y_train)
    print('Training Score : %0.2f'%clf2.score(X_train,y_train))
    print('Testing Score : %0.2f'%clf2.score(X_test,y_test))

Fitting 10 folds for each of 2 candidates, totalling 20 fits


[Parallel(n_jobs=2)]: Done  20 out of  20 | elapsed:    8.6s finished


Training Score : 0.89
Testing Score : 0.77
Fitting 10 folds for each of 2 candidates, totalling 20 fits


[Parallel(n_jobs=2)]: Done  20 out of  20 | elapsed:    9.2s finished


Training Score : 0.88
Testing Score : 0.87
Fitting 10 folds for each of 2 candidates, totalling 20 fits


[Parallel(n_jobs=2)]: Done  20 out of  20 | elapsed:    8.9s finished


Training Score : 0.88
Testing Score : 0.80
Fitting 10 folds for each of 2 candidates, totalling 20 fits


KeyboardInterrupt: 

In [399]:
data_test = pd.read_csv('test.csv')
print('Number Of Rows: {}'.format(len(data_test)))
print('---------------------------------------')
data_test.head(3)

Number Of Rows: 418
---------------------------------------


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q


In [400]:
data_submit = pd.read_csv('gender_submission.csv')
print(len(data_submit.PassengerId))
print(len(data_test.PassengerId))
print(len(data_submit.PassengerId) == len(data_test.PassengerId))
data_submit.head()

418
418
True


,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


In [401]:
data_test.drop(data_test.columns[[2,7,9,10]],axis=1,inplace=True)
len(data_test.PassengerId)
data_test.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare
0,892,3,male,34.5,0,0,7.8292
1,893,3,female,47.0,1,0,7.0000
2,894,2,male,62.0,0,0,9.6875
3,895,3,male,27.0,0,0,8.6625
4,896,3,female,22.0,1,1,12.2875


In [402]:
data_test.Sex = data_test.Sex.astype('category').cat.codes

In [403]:
data_test.groupby('Sex').Age.mean()

Sex
0    30.272362
1    30.272732
Name: Age, dtype: float64

In [404]:
# test_mean_age = mean_age = np.sqrt(np.sum(np.square(data_train.Age.dropna()))/len(data_train.Age.dropna()))
# data_test.fillna(value=test_mean_age,inplace=True)
# data_test.Sex = data_test.Sex.astype('category').cat.codes

# final_X = data_test.as_matrix()
# final_X
print("Average Age Men: %.2f\nAverage Age Women %0.2f"%(data_test[data_test['Sex'] == 0].Age.mean(),data_test[data_test['Sex'] == 1].Age.mean()))
male_mean_age = data_test[data_test['Sex'] == 0].Age.mean()
female_mean_age = data_test[data_test['Sex'] == 1].Age.mean()
# data_train.set_index('Sex',inplace=True)
# data_train.loc[0].Age.fillna(male_mean_age,inplace=True)
# data_train.loc[1].Age.fillna(female_mean_age,inplace=True)
# data_train = data_train.reset_index()
# data_train.isnull().any()
data_test.loc[data_test['Sex'] == 0,'Age'] = data_test.loc[data_test['Sex'] == 0,'Age'].fillna(male_mean_age)
data_test.loc[data_test['Sex'] == 1,'Age'] = data_test.loc[data_test['Sex'] == 1,'Age'].fillna(female_mean_age)
data_test['FamilySize'] = data_test['SibSp'] + data_test['Parch']

# data_test.drop(data_test.columns[[0,4,5]],axis=1,inplace=True)
# data_test.drop(data_test.columns[[0,4,5]],axis=1,inplace=True)

data_test

Average Age Men: 30.27
Average Age Women 30.27


,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,FamilySize
0,892,3,1,34.500000,0,0,7.8292,0
1,893,3,0,47.000000,1,0,7.0000,1
2,894,2,1,62.000000,0,0,9.6875,0
3,895,3,1,27.000000,0,0,8.6625,0
4,896,3,0,22.000000,1,1,12.2875,2
5,897,3,1,14.000000,0,0,9.2250,0
6,898,3,0,30.000000,0,0,7.6292,0
7,899,2,1,26.000000,1,1,29.0000,2
8,900,3,0,18.000000,0,0,7.2292,0
9,901,3,1,21.000000,2,0,24.1500,2


In [405]:
data_test.Pclass.unique()

array([3, 2, 1])

In [406]:
class1_mean = data_test[data_test['Pclass'] == 1].Fare.mean()
class2_mean = data_test[data_test['Pclass'] == 2].Fare.mean()
class3_mean = data_test[data_test['Pclass'] == 3].Fare.mean()
print(class1_mean,class2_mean,class3_mean)
data_test.loc[data_test['Pclass'] == 1,'Fare'] = data_test.loc[data_test['Pclass'] == 1,'Fare'].fillna(class1_mean)
data_test.loc[data_test['Pclass'] == 2,'Fare'] = data_test.loc[data_test['Pclass'] == 2,'Fare'].fillna(class2_mean)
data_test.loc[data_test['Pclass'] == 3,'Fare'] = data_test.loc[data_test['Pclass'] == 3,'Fare'].fillna(class3_mean)
data_test.isnull().any()

94.28029719626169 22.20210430107527 12.459677880184334


PassengerId    False
Pclass         False
Sex            False
Age            False
SibSp          False
Parch          False
Fare           False
FamilySize     False
dtype: bool

In [407]:
data_test['Pclass X Fare'] = data_test['Pclass'] * data_test['Fare']
data_test['Pclass X Fare X Age'] = data_test['Pclass X Fare'] * data_test['Age']
data_test.drop(['PassengerId','Parch','SibSp','Pclass X Fare','Age'],axis=1,inplace=True)

In [408]:
final_X = data_test.as_matrix()
final_X

array([[  3.00000000e+00,   1.00000000e+00,   7.82920000e+00,
          0.00000000e+00,   8.10322200e+02],
       [  3.00000000e+00,   0.00000000e+00,   7.00000000e+00,
          1.00000000e+00,   9.87000000e+02],
       [  2.00000000e+00,   1.00000000e+00,   9.68750000e+00,
          0.00000000e+00,   1.20125000e+03],
       ..., 
       [  3.00000000e+00,   1.00000000e+00,   7.25000000e+00,
          0.00000000e+00,   8.37375000e+02],
       [  3.00000000e+00,   1.00000000e+00,   8.05000000e+00,
          0.00000000e+00,   7.31086471e+02],
       [  3.00000000e+00,   1.00000000e+00,   2.23583000e+01,
          2.00000000e+00,   2.03054045e+03]])

In [409]:
clf2.best_estimator_.score(x_ts,y_ts)

0.88789237668161436

In [410]:
final_y = clf2.predict(final_X)
final_y.shape

(418,)

In [411]:
data_submit['Survived_'] = final_y
del data_submit['Survived']
data_submit.rename(columns = {'Survived_':'Survived'},inplace=True)
data_submit.shape

(418, 2)

In [412]:
data_submit.set_index('PassengerId')

,Survived
PassengerId,
892,0
893,0
894,0
895,0
896,0
897,0
898,0
899,0
900,1


In [413]:
data_submit.to_csv('submission.csv',index=False)